In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=ab765f740ac4317cd0b8df3b156ed21c55910da59303359c9ee7e911ebe970b6
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/wansitorres/FreightMate/refs/heads/main/Freightdata.csv')

In [4]:
df.sample(5)

,Order ID,Item Description,Weight (kg),Volume (m³),Origin City,Destination City,Urgency Level,Preferred Delivery Time (days),Distance (km),Budget (PHP),Special Requirements,Freight Mode Used,Reason for Freight Mode Chosen
8,9,Books,300,1.2,Manila,Bacolod City,Medium,5,610,"4,000",Non-urgent shipping,Sea,Cost-effective option for non-perishable items
15,16,Frozen Meat,500,1.0,Cebu City,Zamboanga City,High,2,820,"12,000",Refrigerated,Sea,Sea freight used due to high refrigeration cap...
5,6,Clothing,200,0.8,Manila,Cebu City,Low,5,800,"6,000",Regular shipping,Sea,Sea offered lower cost for low-priority items
4,5,Fresh Vegetables,150,0.5,Benguet,Quezon City,High,1,300,"5,000",Requires refrigeration,Land,Land chosen due to proximity and rapid deliver...
7,8,Medical Supplies,50,0.1,Manila,Cagayan de Oro,High,1,790,"10,000","High priority, fragile",Air,Air used to ensure timely and safe delivery of...


## Understanding the Dataset

In [16]:
# Get Column Names and Data Types

column_info = df.dtypes
df['Order ID'] = df['Order ID'].astype(object)
print(column_info, "\n")

Order ID                           object
Item Description                   object
Weight (kg)                         int64
Volume (m³)                       float64
Origin City                        object
Destination City                   object
Urgency Level                      object
Preferred Delivery Time (days)      int64
Distance (km)                      object
Budget (PHP)                       object
Special Requirements               object
Freight Mode Used                  object
Reason for Freight Mode Chosen     object
dtype: object 



In [17]:
# Identify Categorical and Numerical Features

categorical_features = df.select_dtypes(include=['object']).columns.tolist()
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)

Categorical Features: ['Order ID', 'Item Description', 'Origin City', 'Destination City', 'Urgency Level', 'Distance (km)', 'Budget (PHP)', 'Special Requirements', 'Freight Mode Used', 'Reason for Freight Mode Chosen']
Numerical Features: ['Weight (kg)', 'Volume (m³)', 'Preferred Delivery Time (days)']


In [18]:
# Get Unique Values for Categorical Features with Limitation

def get_unique_values(col, max_display= 10):
  unique_values= df[col].unique().tolist()
  if len(unique_values) <= max_display:
    return unique_values[:max_display] + ["..."]
  return unique_values
unique_values = {col: get_unique_values(col) for col in categorical_features}
print("Unique Values for Categorical Features:")
for col, values in unique_values.items():
  print(f"- {col}: {values}")

Unique Values for Categorical Features:
- Order ID: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
- Item Description: ['Fresh Tuna', 'Electronics Parts', 'Rice', 'Furniture Set', 'Fresh Vegetables', 'Clothing', 'Construction Materials', 'Medical Supplies', 'Books', 'Livestock Feed', 'Pharmaceuticals', 'Household Appliances', 'Industrial Machinery', 'Office Supplies', 'High-end Perfumes', 'Frozen Meat', 'Handcrafted Furniture', 'Fresh Flowers', 'Seafood', 'Agricultural Equipment']
- Origin City: ['General Santos', 'Manila', 'Nueva Ecija', 'Cebu City', 'Benguet', 'Davao City', 'Bacolod City', 'Pampanga', 'Baguio City', 'Batangas', '...']
- Destination City: ['Cebu City', 'Davao City', 'Iloilo City', 'Manila', 'Quezon City', 'Cagayan de Oro', 'Bacolod City', 'Zamboanga City', '...']
- Urgency Level: ['High', 'Medium', 'Low', '...']
- Distance (km): ['700', '960', '450', '570', '300', '800', '790', '610', '400', '820', '1,000', '250', '1,100', '1,200']
- Budget (P

In [19]:
# Get Descriptive Statistics for Numerical Features

numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")


Descriptive Statistics for Numerical Features:
        Weight (kg)  Volume (m³)  Preferred Delivery Time (days)
count    20.000000    20.000000                       20.000000
mean   1498.500000     2.612500                        4.300000
std    2417.347992     3.791676                        3.599708
min      20.000000     0.050000                        1.000000
25%     100.000000     0.275000                        1.000000
50%     300.000000     1.100000                        4.000000
75%    1100.000000     3.250000                        6.000000
max    8000.000000    15.000000                       15.000000 



## Generating the Template

In [28]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f"- {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, values in unique_values.items():
        unique_values_str += f"- {col}: {values}\n"


    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:  # Removed the parentheses here
        numerical_stats_str += f"- {col}:\n"
        for stat, value in numerical_stats[col].items():
            numerical_stats_str += f"  {stat}: {value}\n"


    # Define the system prompt
    system_prompt = """
    "You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness
    """


    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
      try:
        response = openai.ChatCompletion.create(
            model= "gpt-4o-mini",
            temperature = 0.3,
            max_tokens = 1024,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        template = response['choices'][0]['message']['content'].strip()
        return template
      except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)
    return None


In [29]:
# Generate the template

template = generate_template(df,
    column_info,
    categorical_features,
    numerical_features,
    unique_values,
    numerical_stats)

print(template)

"Order ID {Order ID} involves the shipment of {Item Description}, weighing {Weight (kg)} kg and occupying a volume of {Volume (m³)} m³. The shipment originates from {Origin City} and is destined for {Destination City}. Given the {Urgency Level} urgency level, a preferred delivery time of {Preferred Delivery Time (days)} days has been set, covering a distance of {Distance (km)} km. The budget allocated for this order is {Budget (PHP)}, with special requirements noted as '{Special Requirements}'. The freight mode selected for this shipment is {Freight Mode Used}, chosen for the following reason: '{Reason for Freight Mode Chosen}'."


## Populating the Template with Actual Values

In [34]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph

In [35]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

In [36]:
df['paragraph'][0]

'"Order ID 1 involves the shipment of Fresh Tuna, weighing 200 kg and occupying a volume of 0.8 m³. The shipment originates from General Santos and is destined for Cebu City. Given the High urgency level, a preferred delivery time of 1 days has been set, covering a distance of 700 km. The budget allocated for this order is 15,000, with special requirements noted as \'Requires refrigeration\'. The freight mode selected for this shipment is Air, chosen for the following reason: \'Chose air for speed due to perishable goods\'."'

## Saving the Populated Templates (Natural Language Texts)

In [39]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [40]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")


In [42]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].tolist(), txt_filename='paragraphs.txt')
save_content_to_pdf(df['paragraph'].tolist(), pdf_filename='paragraphs.pdf')


All content has been saved as 'paragraphs.txt'.

All content has been saved as 'paragraphs.pdf'.

